In [1]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, select_shares, private_compare
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor

import unittest
import numpy as np
import torch
import importlib


In [2]:
hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [48]:
def decompose(tensor):
    """
    decompose a tensor into its binary representation
    """
    powers = torch.arange(0,Q_BITS)
    for i in range(len(tensor.shape)):
        powers = powers.unsqueeze(0)
    tensor = tensor.unsqueeze(-1)
    moduli = 2 ** powers
    tensor = ((tensor+2**(Q_BITS)) / moduli.type_as(tensor)) % 2
    return tensor

def _pc_beta0(x, r):
    # note x and r are both binary tensors,
    # and dim -1 contains their bits
    # x should be shared, r should be public
    w = xor(x, r)
    print(w.shape)
    z = r  - (x - 1)
    w_sum = torch.zeros(w.shape).type_as(w)
    for i in range(Q_BITS - 1, -1, -1):
        # FIXME: double check if keepdim should be True/False here
        w_sum[..., i] = w[..., (i + 0):].sum(dim=-1, keepdim=True)
    c = z + w_sum
    return c


def _pc_beta1(x, t):
    pass


def _pc_else():
    pass


def xor(x, y):
    return x + y - 2 * x * y

In [49]:
x = torch.LongTensor([[5]])#.share(bob, alice)

r = torch.LongTensor([[4]])

Q_BITS = spdz.Q_BITS

beta = (torch.rand(1)>0).long().unsqueeze(0)

t = (r + 1) % (2 ** Q_BITS)

x_bits = decompose(x)
r_bits = decompose(r)
t_bits = decompose(t)

zeros = (beta == 0).long()
ones = (beta == 1).long()
others = (r == (2 ** Q_BITS - 1)).long()
ones = ones & (others - 1).abs()

In [55]:
c_zeros = _pc_beta0(x_bits, r_bits)
c_ones = _pc_beta1(x_bits, t_bits)
c_other = _pc_else()

torch.Size([1, 1, 31])


In [56]:
c_zeros


(0 ,.,.) = 

Columns 0 to 18 
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 19 to 30 
    1   1   1   1   1   1   1   1   1   1   1   1
[syft.core.frameworks.torch.tensor.LongTensor of size 1x1x31]

In [57]:
c_ones

In [58]:
c = torch.cat([c_zeros, c_ones, c_other], -1)

TypeError: cat received an invalid combination of arguments - got (list, int), but expected one of:
 * (sequence[torch.LongTensor] seq)
 * (sequence[torch.LongTensor] seq, int dim)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m, [32;1mint[0m)


In [51]:
# TODO: recombine c properly here
# torch.zeros()


s = random_as(c, mod=p)
permute = torch.randperm(c.size(-1))
d = s * c[..., permute]
d.get()

TypeError: cat received an invalid combination of arguments - got (list, int), but expected one of:
 * (sequence[torch.LongTensor] seq)
 * (sequence[torch.LongTensor] seq, int dim)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m, [32;1mint[0m)


In [66]:
def private_compare(x, r, beta, workers):
    """
    computes beta XOR (x > r)

    x is private input
    r is public input for comparison
    beta is public random bit tensor

    all of type _GeneralizedPointerTensor
    """
    t = (r + 1) % (2 ** Q_BITS)

    x_bits = decompose(x)
    r_bits = decompose(r)
    t_bits = decompose(t)

    zeros = beta == 0
    ones = beta == 1
    others = r == (2 ** Q_BITS - 1)
    ones = ones & (others - 1).abs()

    c_zeros = _pc_beta0(x_bits, r_bits)
    c_ones = _pc_beta1(x_bits, t_bits)
    c_other = _pc_else()

    # TODO: recombine c properly here
    # torch.zeros()
    c = torch.cat([c_zeros, c_ones, c_other], -1)

    s = random_as(c, mod=p)
    permute = torch.randperm(c.size(-1))
    d = s * c[..., permute]
    d.get()
    return (d == 0).max()